In [ ]:
def generate_channels_for_doppler_frequencies(Fd, Fs, N):
    # Convert Fd to array if single value
    channels = np.zeros(N, dtype=complex)
    # Design Jakes filter
    filter_length = 512  # Filter length
    L = filter_length // 2
    Ts = 1/Fs

    # Generate filter coefficients
    n = np.arange(1, L+1)
    # Positive frequencies part
    J_pos = special.jv(0.25, 2*np.pi*Fd*n*Ts) / (n**0.25)
    # Negative frequencies part (mirror of positive)
    J_neg = np.flip(J_pos)
    # Center coefficient
    J_0 = 1.468813 * (Fd*Ts)**0.25

    # Combine all parts
    J = np.concatenate([J_neg, [J_0], J_pos])

    # Apply Hamming window
    n_window = np.arange(filter_length + 1)
    hamming = 0.54 - 0.46 * np.cos(2*np.pi*n_window/filter_length)
    hw = J * hamming

    # Normalize filter
    hw = hw / np.sqrt(np.sum(np.abs(hw)**2))

    # Generate complex Gaussian noise
    noise_real = np.random.normal(0, 1/np.sqrt(2), N + len(hw) - 1)
    noise_imag = np.random.normal(0, 1/np.sqrt(2), N + len(hw) - 1)
    complex_noise = noise_real + 1j*noise_imag

    # Filter the noise
    filtered_signal = signal.convolve(complex_noise, hw, mode='valid')

    # Store the channel values
    channels = filtered_signal[:N]

    return channels